In [ ]:
import pandas as pd
import catboost as cb
import numpy as np
import os

from catboost import Pool
from sklearn.model_selection import train_test_split

SEED = 5
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(SEED)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/moscow-metro/data/train_dataset_train.csv', parse_dates=['pass_dttm'])
df_test = pd.read_csv('/content/drive/MyDrive/moscow-metro/data/test_dataset_test.csv', parse_dates=['pass_dttm'])

In [ ]:
def fe(df):
    '''
    Generate features
    '''
    df['day'] = df['pass_dttm'].dt.day
    df['weekday'] = df['pass_dttm'].dt.weekday
    df['seconds'] = (df['pass_dttm'].dt.hour * 60 + df['pass_dttm'].dt.minute) * 60 + df['pass_dttm'].dt.second

    df = df.drop(['pass_dttm'], axis=1)
    return df

In [ ]:
train = fe(df).drop(["ticket_id", "entrance_nm", "station_nm", "line_nm", "label", "time_to_under", 'id'], axis = 1)
test = fe(df_test).drop(["ticket_id", "entrance_nm", "station_nm", "line_nm", 'id'], axis = 1)

In [ ]:
cat_features = range(0,6)
X = train
y_clf = df["label"]

train_pool = Pool(X, y_clf, cat_features=cat_features)

clf = cb.CatBoostClassifier(loss_function='MultiClassOneVsAll',
                            task_type="GPU",
                            learning_rate=0.05,
                            depth=7,
                            iterations=3236,
                            verbose=200,
                            random_state=SEED
                            )
clf.fit(train_pool)
clf_pred = clf.predict(test)

In [ ]:
X = train
y_reg = df["time_to_under"]

train_pool = Pool(X, y_reg, cat_features=cat_features)

reg = cb.CatBoostRegressor(task_type="GPU",
                           learning_rate=0.05,
                           depth=8,
                           verbose=200,
                           iterations=1597,
                           random_state=SEED,
                           )
reg.fit(train_pool)
reg_pred = reg.predict(test)

In [ ]:
sample_solution = pd.read_csv('/content/drive/MyDrive/moscow-metro/subway/sample_solution.csv')
sample_solution['time_to_under'] = reg_pred
sample_solution['label'] = clf_pred
sample_solution.to_csv('half.csv', index=False)